<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
      Using the Script UI in Teradataml.
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;'>The Teradataml Widgets (teradatamlwidgets) enhances teradataml’s built-in interaction capabilities with the Teradata Vantage™ Data and Analytics Platform. This provides  visual components for scaled, in-Database Analytics with data that you keep in the Teradata Vantage Analytics Database within a notebook.</p>

<p style = 'font-size:16px;font-family:Arial;'>With these components, in this notebook you will be able to:</p>

<ul style = 'font-size:16px;font-family:Arial;'>
    <li>Script Table Operator function</li>
</ul>

<hr style="height:2px;border:none;">
<p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>Install/update packages - Do this if you have any issues executing the cells in this notebook.</b></p>
 <p style = 'font-size:16px;font-family:Arial;color:#E37C4D'><b>NOTE: If you update the teradatamlwidgets library, you will need to restart the Kernel.</b></p> 

In [ ]:
# !pip install teradatamlwidgets ipywidgets --quiet

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Log into the database.</b></p>
<ul style = 'font-size:16px;font-family:Arial;'>
    <li>Host = host.docker.internal</li>
    <li>Username = demo_user</li>
    <li>Password = the password you entered when you created this environment.</li>
</ul>
<p style = 'font-size:16px;font-family:Arial'>Then click the <b>Login</b> button.</p>

In [ ]:
from teradatamlwidgets import login,script

ui = login.Ui()

In [ ]:
import getpass
from teradataml import *


display.max_rows = 5
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage.  You have the option of either running the demo using foreign tables to access the data without using any storage on your environment or downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage.  There are two statements in the following cell, and one is commented out.  You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_HyperModel_cloud');"
 # Takes about 20 seconds
#%run -i ../../UseCases/run_procedure.py "call get_data('DEMO_HyperModel_local');"
 # Takes about 50 secs

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>2. The Dataset</b></p>
<p style = 'font-size:16px;font-family:Arial'>Let us take a look at the sample dataset we are using.</p>

In [ ]:
dataset = DataFrame(in_schema('DEMO_HyperModel', 'Dataset'))
dataset

<p style = 'font-size:18px;font-family:Arial'><b>Setting default database</b></p>

In [ ]:
default_database = "demo_user"

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>3. Uploading Files</b></p>
<p style = 'font-size:16px;font-family:Arial'>The below code block will open the Script UI. Please enter the details as below:<br>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Go to the <b>Files</b> tab <br> Enter this value in the <b>ID</b> field: "script_example"<br>
      Enter this value in the <b>Name</b> field: "script_example.py"<br>  
        Change <b>Replace</b> to be selected</li>
    <li>Click the <b>Install</b> button<br>You should receive this message, "File installed in Teradata Vantage Enterprise."<br>
        Click the <b>Close</b> button.

In [ ]:
from teradatamlwidgets import script
ui = script.Ui(
    files_local_path='./scripts', 
    script_command='echo upload')

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>4. Executing the script</b></p>
<p style = 'font-size:16px;font-family:Arial'>The below code block will open the Script UI again. Please follow the below steps to run the script:<br>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>The value for <b>Command</b> going to be: <br><code>tdpython3  ./'+default_database+'/script_example.py</code></li>
    <li>Go to <b>Output</b> tab. You should see these fields already defined:<br>"Partition_ID": BIGINT()<br>"ID": BIGINT()<br>"ANOMALY": BYTEINT()<br>"DECISION_FUNCTION": FLOAT()<br>"ANOMALY_SCORE": FLOAT()<br></li>
<li>Click the <b>Execute</b> button.

In [ ]:
ui = script.Ui(
    data=dataset[dataset.FOLD == 'train'],
    script_name='execute',
    files_local_path='.', 
    script_command='tdpython3  ./'+default_database+'/script_example.py',
    search_path=default_database,
    data_order_column     = ["ID"],
    data_partition_column = ['Partition_ID'],
    is_local_order        = False,
    nulls_first           = False,
    sort_ascending        = False,
    charset               = 'latin',
    returns={"Partition_ID": BIGINT(), "ID": BIGINT(), "ANOMALY": BYTEINT(), "DECISION_FUNCTION": FLOAT(),"ANOMALY_SCORE": FLOAT()})

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>5. Get the output datadrame</b></p>
<p style = 'font-size:16px;font-family:Arial'>Execute the following cell to see the output dataframe</p>

In [ ]:
ui.get_output_dataframe()

<hr style="height:2px;border:none;">

<p style = 'font-size:20px;font-family:Arial'><b>6. Removing Files</b></p>
<p style = 'font-size:16px;font-family:Arial'>The below code block will remove the files loaded</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Go to Files</li>
    <li>Set ID to <b>script_example</b> turn off Binary, Turn on Force and Click Remove</li>

In [ ]:
ui = script.Ui(
    files_local_path='./scripts', 
    script_command='echo remove')

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>7. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../../UseCases/run_procedure.py "call remove_data('DEMO_HyperModel');" 
#Takes 40 seconds

In [ ]:
remove_context()

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2026. All Rights Reserved
        </div>
    </div>
</footer>